In [1]:
from lxml import html
import requests
from pprint import pprint
from pymongo import MongoClient
from pprint import pprint

In [2]:
url = 'https://news.mail.ru/'

In [3]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}

In [4]:
response = requests.get(url, headers=headers)

In [5]:
dom = html.fromstring(response.text)

In [6]:
item_list = []

In [7]:
# the main article
item_info = {}
name = dom.xpath('//span[contains(@class, "photo__title_new")]//text()')[0].replace('\xa0', ' ')
link = dom.xpath('//td[@class="daynews__main"]//@href')[0]
response_main = requests.get(link, headers=headers)
dom_1 = html.fromstring(response_main.text)
date = dom_1.xpath('//span/@datetime')[0]
source = dom_1.xpath('//span[@class="note"]/*/span[@class="link__text"]//text()')[0]

item_info['name'] = name
item_info['link'] = link
item_info['date'] = date
item_info['source'] = source
item_list.append(item_info)

In [8]:
items = dom.xpath('//div[@class="daynews__item"]')

In [9]:
# articles with a photo
for item in items:
    item_info = {}
    name = item.xpath('.//span[contains(@class, "photo__title_new")]//text()')[0].replace('\xa0', ' ')
    link = item.xpath('.//@href')[0]
    response_main = requests.get(link, headers=headers)
    dom_1 = html.fromstring(response_main.text)
    date = dom_1.xpath('//span/@datetime')[0]
    source = dom_1.xpath('//span[@class="note"]/*/span[@class="link__text"]//text()')[0]
    
    item_info['name'] = name
    item_info['link'] = link
    item_info['date'] = date
    item_info['source'] = source
    item_list.append(item_info)
    
    

In [10]:
items_below = dom.xpath('//ul[contains(@class, "list_half")]/li[@class="list__item"]')

In [ ]:
# articles without photos
for item in items_below:
    item_info = {}
    name = item.xpath('./a//text()')[0].replace('\xa0', ' ')
    link = item.xpath('.//@href')[0]
    response_main = requests.get(link, headers=headers)
    dom_1 = html.fromstring(response_main.text)
    date = dom_1.xpath('//span/@datetime')[0]
    source = dom_1.xpath('//span[@class="note"]/*/span[@class="link__text"]//text()')[0]
    
    item_info['name'] = name
    item_info['link'] = link
    item_info['date'] = date
    item_info['source'] = source
    item_list.append(item_info)

In [ ]:
item_list

In [ ]:
client = MongoClient('127.0.0.1', 27017)

In [ ]:
db = client['articles']

In [ ]:
mailru = db.mailru

In [ ]:
mailru.insert_many(item_list)

In [ ]:
item_list